# Projeto 2 - Como pegar dados de um site com Python? - Pegando dados de ETFs do mundo inteiro.
**Desafio:**
Construir um código que vá no site etf.com e busque dados de todos os etfs do mercado americano e, consequentemente, do mundo. Rentabilidades, patrimônio, gestora, taxa...
Lembrar de sempre trazer outros tipos de cenários onde a pessoa precisa pegar dados de sites etc.

**Passo a passo:**

**Passo 1** - Definir um navegador que você irá utilizar para navegar com o Python.

**Passo  2** - Importar os módulos e bibliotecas.

**Passo 3** - Entender como funcionam requisições na internet.

**Passo 4** - Conhecer e mapear o processo de coleta de dados no site do ETF.com.

**Passo 5** - Achar todos os elementos necessários dentro do HTML do site.

**Passo 6**- Ler a tabela de dados.

**Passo 7** - Construir a tabela final.


# Passo 1: Escolher o navegador.

Usaremos o Google Chrome.

# Passo 2: Importar as bibliotecas.

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

# Passo 3: Entender como funcionam requisições na internet.

In [3]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

# Passo 4: Conhecer e mapear o processo de coleta de dados no site do ETF.COM. 

Neste caso, deve-se entrar no site e entender como o processo seria feito de maneira manual, para assim entender como a automação deve ser feita.

# Passo 5.1: Achar todos os elementos necessários dentro do HTML do site -Expandindo a tabela para 100 itens. 

In [4]:
time.sleep(5)

botao_100 = driver.find_element('xpath',
                                '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

driver.execute_script('arguments[0].click();', botao_100)
#botao_100.click()

# Passo 5.2: Achar todos os elementos necessários dentro do HTML do site - Pegando o número de páginas da tabela.

In [5]:
numero_paginas_element = driver.find_element('xpath', '//*[@id="totalPages"]')
numero_paginas_text = numero_paginas_element.text
match = re.search(r'(\d+)', numero_paginas_text)
if match:
    numero_paginas = int(match.group(1))
    print(numero_paginas)
else:
    print('Não foi possível extrair o numero de páginas.')

32


# Passo 6.1: Lendo a tabela de dados - Lendo a tabela de dados básicos.

In [6]:
elemento = driver.find_element('xpath', '//*[@id="finderTable"]')
html_tabela = elemento.get_attribute('outerHTML')
tabela = pd.read_html(str(html_tabela))[0]

tabela

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B
...,...,...,...,...,...,...
95,VXF,Vanguard Extended Market ETF,Equity: U.S. - Extended Market,Vanguard,0.06%,$14.04B
96,BIV,Vanguard Intermediate-Term Bond ETF,"Fixed Income: U.S. - Broad Market, Broad-based...",Vanguard,0.04%,$14.02B
97,XLI,Industrial Select Sector SPDR Fund,Equity: U.S. Industrials,State Street Global Advisors,0.10%,$13.89B
98,HYG,iShares iBoxx USD High Yield Corporate Bond ETF,"Fixed Income: U.S. - Corporate, Broad-based Hi...",Blackrock,0.48%,$13.73B


In [8]:
lista_tabela_por_pagina = []

elemento = driver.find_element('xpath', '//*[@id="finderTable"]')


for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_avancar_pagina = driver.find_element('xpath', '//*[@id="nextPage"]')
    
    driver.execute_script('arguments[0].click();', botao_avancar_pagina)
    

tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)

tabela_cadastro_etfs

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B
...,...,...,...,...,...,...
20,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.15%,--
21,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
22,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,0.18%,--
23,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",Equity: U.S. - Large Cap,Morgan Stanley,0.14%,--


# Passo 6.2: Ler a tabela de dados - Preenchendo um campo no site pra voltar as páginas

In [11]:
formulario_de_voltar_pagina = driver.find_element('xpath', '//*[@id="goToPage"]')

formulario_de_voltar_pagina.clear()
formulario_de_voltar_pagina.send_keys('1')
formulario_de_voltar_pagina.send_keys(u'\ue007')